# Sea Level Data Scrub and Linear Regression
* This Takes the original Source data from NASA and creates a text file that can be used by the other members on the team for correlation analysis.
* Visualizations show the initial scrub of the full data and the scrub of the grouped data (when looking at the full data the sea level is rising grouped and only limited to 2015 the picture looks like sea level is dropping)

In [ ]:
# Necessary Imports 
%matplotlib inline 

import requests 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.linalg as lin
from datetime import timedelta, datetime
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from datetime import timedelta, datetime
from sklearn.linear_model import LinearRegression
from io import StringIO

In [ ]:
sea_data = pd.read_csv('data/GMSL_merged_nasa_1993_2020.csv')
# sea_data 
sea_data.head(5)

# sea_level = sea_data['GMSL'].values
# sea_level

In [ ]:
#conversion formula for Converting year fractions aka year decimals into datetimes
def convert_partial_year(number):
    year = int(number)
    d = timedelta(days = (number - year) * 365.25)
    day_one = datetime(year, 1, 1)
    date = d + day_one
    return date

In [ ]:
#Get Results
sea_data['date'] = sea_data['year_fraction'].apply(convert_partial_year)
#sea_data

#Drop the unused column
sea_data_1 = sea_data.drop(['year_fraction'], axis=1)
sea_data_1 

In [ ]:
#Get the Year
sea_data_1['year'] = sea_data_1['date'].apply(lambda x: x.year)
#sea_data_1

#Drop unused date
sea_data_2 = sea_data_1.drop(['date'], axis=1)
sea_data_2

# Re-order Columns
sea_data_2 = sea_data_2[['year', 'gmsl']]
sea_data_2.tail(5) 

In [ ]:
#Rename Columns 
sea_data_2.columns = ['Year', 'GMSL']
sea_data_2

#Dropping off outliers index row 9 for 1993 , 2018 945-953 and for 2019 remove 980-993
# sea_data_drop = sea_data_2.drop(sea_data_2.index[9])
# sea_data_drop

# #Drop the 2018 945-953
# sea_data_3 = sea_data_drop.drop(sea_data_drop.index[944:953])
# sea_data_3.head(5)

In [ ]:
#Group by and reset index 
# Drop everything after 2015 
#Find average gmsl after grouping 

sea_data_3 = sea_data_2.groupby('Year')['GMSL'].mean()
sea_data_3.reset_index()

#Drop everything after 2015
sea_data_4 = sea_data_3.drop(sea_data_3.index[23:28])

In [ ]:
#Data 
data = sea_data_3.reset_index() # load data set  
X = data.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
Y = data.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions

In [ ]:
#Plot 
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.xlabel('Years')
plt.ylabel('Global Mean Sea Level (GMSL) in (mm)')
plt.title('Sea Level over Time (in mm)')
#Save it
plt.savefig('images/sealevel_scrub_fig1.png')

In [ ]:
#Reset Index 
sea_level_data = sea_data_4.reset_index()
sea_level_data

In [ ]:
#Data 
data = sea_level_data # load data set  
X = data.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
Y = data.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)  # make predictions

In [ ]:
#Plot 
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.xlabel('Years')
plt.ylabel('Global Mean Sea Level (GMSL) in (mm)')
plt.title('Sea Level over Time (in mm)')


#Save it
plt.savefig('images/sealevel_scrub_fig2.png')

The sea level from 1993 - 2015 has some outliers and I decided to keep the outliers pending furthe research.

The GMSL is a 1-dimensional time series of globally averaged Sea Surface Height Anomalies (SSHA) from TOPEX/Poseidon, Jason-1, OSTM/Jason-2 and Jason-3. It starts in September 1992 to present, with a lag of up to 4 months. All biases and cross-calibrations have been applied to the data so SSHA are consistent between satellites. Data are reported as changes relative to January 1, 1993 and are 2-month averages. Glacial Isostatic Adjustment (GIA) has been applied. These data are available in ASCII format. Reference: Beckley et al., 2017,

This is the “uncertainty margin,” or the range from the mean (average) within which there is a high probability that the true number resides. For example, when we say that global mean sea level for a particular month is 50mm +/- 4 mm, the true value for that month has a high probability of being between 46 and 54 mm. The uncertainty margin exists both because of sea level variability (sea level above the first point in the time series) in the ocean and the accuracy with which we measure the individual values. https://sealevel.nasa.gov/understanding-sea-level/key-indicators/global-mean-sea-level/ Source: https://podaac-tools.jpl.nasa.gov/drive/files/allData/merged_alt/L2/TP_J1_OSTM/global_mean_sea_level/GMSL_TPJAOS_4.2_199209_202001.txt

In [ ]:
#To make a sea level data that can be used for the rest of the team in consolidating the data. 
sea_level_data.to_csv("data/sealevel_data.csv", index = False, header = True)